# PART-1: A full report consisting of all of the given components:

### 1. Introduction where you discuss the business problem and who would be interested in this project:

About the place (Where):
Toronto

Interested personals:
Tourists of Toronto as well as Tourism Agencies

Description:
Toronto is one of the most attractive places for tourists to visit. There are so many people who come to visit there every year, but there are so many who can't understand where to go and about their perspectives. Tourists normally want to explore variety of places when they come to arrive in unexplored region, but tourists also have some kind of inclination towards certain venues over the other venues. I want to build a recommended system on base of tourists' preferences. This is builded on an observation that tourists always try to explore places which are nearby first. If one would arrive in Toronto,he/she will start exploring a particular neighbourhood, where they have stayed and then will start exploring another places.

By keeping in mind this particular thing, I have build this system, which can be helpful to tourism agencies as well as to tourists to make their guide-book for exploring places.

### 2. Data where you describe the data that will be used to solve the problem and the source of the data:

##### Required Data

The required data about different venues in different neighborhoods of that specific borough. In order to gain that information we will use Foursquare location data. Location data for each venue means basic and advanced information about that venue. As basic information, one can obtain the precise latitude and longitude of a specific venue and also its distance from the center of the neighborhood, but we are looking for advanced information such as the category of that venue and whether this venue is a popular one in its category or the average price of the services of this venue as well as ratings given to the venue. We shall also need data of tourists to know about their preferences. 


For these purposes, we need to:
1. get data of neighborhood of Toronto, which we can get from WikiPedia
2. get users' ratings for recommend as per their preferences

##### Data Mining

In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes ;
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.90 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  39.13 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  49.71 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.68 MB/s
vincent-0.4.4- 100% |###################

In [5]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')

My_table = soup.find('table',{'class':'wikitable sortable'})
table1=""
for tr in My_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]
print(table1[:300])

M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M5A,Downtown Toronto,Regent Park
M6A,North York,Lawrence Heights
M6A,North York,Lawrence Manor
M7A,Queen's Park,Not assigned
M8A,Not assigned,Not ass


In [6]:
file=open("toronto.csv","wb")

file.write(bytes(table1,encoding="ascii",errors="ignore"))

8773

In [7]:
import pandas as pd
df = pd.read_csv('toronto.csv',header=None)
df.columns=["Postalcode","Borough","Neighbourhood"]
df.head(10)

,Postalcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


##### Data Preprocessing

In [8]:
indexNames = df[ df['Borough'] =='Not assigned'].index

df.drop(indexNames , inplace=True)

In [9]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']

In [10]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new=result.reset_index()
df_new.head(15)

,Postalcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


In [11]:
df_new.shape

(103, 3)

In [12]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
df_lon_lat.columns=['Postalcode','Latitude','Longitude']

In [15]:
df_toronto = pd.merge(df_new,
                 df_lon_lat[['Postalcode','Latitude', 'Longitude']],
                 on='Postalcode')
df_toronto.head(10)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


##### Map of Toronto with latitude and longitude

In [16]:
from geopy.geocoders import Nominatim
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [17]:
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 11)


for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

### 3. Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why:

In [18]:
North_York_data = df_toronto[df_toronto['Borough'] == 'North York']
North_York_data = North_York_data.reset_index()
North_York_data.drop('index', axis=1, inplace=True)
North_York_data

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073
5,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923
6,M2H,North York,Hillcrest Village,43.803762,-79.363452
7,M3H,North York,"Bathurst Manor, Downsview North, Wilson Heights",43.754328,-79.442259
8,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
9,M3J,North York,"Northwood Park, York University",43.767980,-79.487262


In [19]:
CLIENT_ID = 'NFLBPPZYATNRIWXWNQVLNMCLVODKFGPBZYJNATZUCEPZIDCJ' 
CLIENT_SECRET = '0XFLFYVFSVPDSXZGAEBXLCOMIXGFPW1XFPO4VILM4P52FL0M' 
VERSION = '20180605' 

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: NFLBPPZYATNRIWXWNQVLNMCLVODKFGPBZYJNATZUCEPZIDCJ
CLIENT_SECRET:0XFLFYVFSVPDSXZGAEBXLCOMIXGFPW1XFPO4VILM4P52FL0M


In [21]:
LIMIT = 100
radius = 500

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'id',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(names=North_York_data['Neighbourhood'],
                                   latitudes=North_York_data['Latitude'],
                                   longitudes=North_York_data['Longitude'])

In [23]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,id,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,4e8d9dcdd5fbbbb6b3003c7b,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,4e6696b6d16433b9ffff47c3,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Careful & Reliable Painting,58d5b8d0102f4722b70e487b,43.752622,-79.331957,Construction & Landscaping
3,Parkwoods,43.753259,-79.329656,Variety Store,4cb11e2075ebb60cd1c4caad,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,4c633acb86b6be9a61268e34,43.723481,-79.315635,Hockey Arena


* Getting ratings of these venues:

In [24]:
rating_list=[]
result_list=[]
c=0
for i in toronto_venues['id']:
    if c<10:
        
        url1='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
        results1 = requests.get(url1).json()#['response']['venue']['rating']
        
        result_list.append(results1)
        try:
            rating_list.append(results1['response']['venue']['rating'])
        except:
            rating_list.append("not rated")
        
    else:
        break

In [25]:
lol = pd.DataFrame(rating_list)

In [26]:
lol.columns=['ratings']

In [27]:
lol.to_csv('ratings.csv')

In [28]:
df_ra = pd.read_csv('ratings.csv')

df_ra.head(10)

,Unnamed: 0,ratings
0,0,7.0
1,1,6.0
2,2,not rated
3,3,not rated
4,4,7.4
5,5,6.3
6,6,6.2
7,7,not rated
8,8,7.8
9,9,7.9


In [29]:
toronto_venues['rating']=df_ra['ratings']

In [30]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,id,Venue Latitude,Venue Longitude,Venue Category,rating
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,4e8d9dcdd5fbbbb6b3003c7b,43.751976,-79.332140,Park,7.0
1,Parkwoods,43.753259,-79.329656,KFC,4e6696b6d16433b9ffff47c3,43.754387,-79.333021,Fast Food Restaurant,6.0
2,Parkwoods,43.753259,-79.329656,Careful & Reliable Painting,58d5b8d0102f4722b70e487b,43.752622,-79.331957,Construction & Landscaping,not rated
3,Parkwoods,43.753259,-79.329656,Variety Store,4cb11e2075ebb60cd1c4caad,43.751974,-79.333114,Food & Drink Shop,not rated
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,4c633acb86b6be9a61268e34,43.723481,-79.315635,Hockey Arena,7.4


In [31]:
final_data = toronto_venues[toronto_venues['rating'] != 'not rated']
final_data.reset_index(inplace=True)
final_data=final_data.drop(['index'],axis=1)
final_data

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,id,Venue Latitude,Venue Longitude,Venue Category,rating
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,4e8d9dcdd5fbbbb6b3003c7b,43.751976,-79.332140,Park,7.0
1,Parkwoods,43.753259,-79.329656,KFC,4e6696b6d16433b9ffff47c3,43.754387,-79.333021,Fast Food Restaurant,6.0
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,4c633acb86b6be9a61268e34,43.723481,-79.315635,Hockey Arena,7.4
3,Victoria Village,43.725882,-79.315572,Portugril,4f3ecce6e4b0587016b6f30d,43.725819,-79.312785,Portuguese Restaurant,6.3
4,Victoria Village,43.725882,-79.315572,Tim Hortons,4bbe904a85fbb713420d7167,43.725517,-79.313103,Coffee Shop,6.2
5,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Roots,4b16e8b6f964a52051bf23e3,43.718476,-79.466869,Boutique,7.8
6,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Lac Vien Vietnamese Restaurant,4ccc5aebee23a14370591ea8,43.721259,-79.468472,Vietnamese Restaurant,7.9
7,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Kitchen Stuff Plus (Clearance Outlet),4b12e300f964a520299023e3,43.719096,-79.462675,Furniture / Home Store,7.0
8,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Orfus Road Shopping Outlets,4bf5b47594b2a593c623acee,43.719045,-79.460849,Clothing Store,7.0
9,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,Tim Hortons,4fa4ae40e4b02443361f27c4,43.719427,-79.467995,Coffee Shop,6.7


##### Analyzing each neighborhood

In [32]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot.head()

,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parkwoods
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Victoria Village


In [33]:
Neig_grouped=toronto_onehot.groupby(['Neighbourhood'], sort=False).sum()

In [34]:
Neig_grouped

,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bike Shop,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Cafeteria,Café,Candy Store,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Electronics Store,Empanada Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,Food Truck,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hockey Arena,Home Service,Hotel,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Massage Studio,Mediterranean Restaurant,Metro Station,Middle Eastern Restaurant,Miscellaneous Shop,Movie Theater,Moving Target,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Lawrence Heights, Lawrence Manor",1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
Don Mills North,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Glencairn,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Flemingdon Park, Don Mills South",0,0,0,0,2,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,2,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
Hillcrest Village,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"Bathurst Manor, Downsview North, Wilson Heights",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
"Fairview, Henry Farm, Oriole",0,0,1,0,2,0,2,1,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,0,1,0,1,10,4,0,0,1,1,1,1,0,0,0,0,1,0,0,6,0,2,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,1,0,1,1,1,0,1,0,1,0,1,2,1,0,0,1,2


In [35]:
Neig_grouped.shape

(23, 105)

##### Dummy user data for making preferences

In [36]:
Central_toronto_data = df_toronto[df_toronto['Borough'] == 'Central Toronto']
Central_toronto_data = Central_toronto_data.reset_index()
Central_toronto_data.drop('index', axis=1, inplace=True)
Central_toronto_data

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
5,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
6,M4S,Central Toronto,Davisville,43.704324,-79.388790
7,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
8,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [37]:
user_rated_venues = getNearbyVenues(names=Central_toronto_data['Neighbourhood'],
                                   latitudes=Central_toronto_data['Latitude'],
                                   longitudes=Central_toronto_data['Longitude'])

In [38]:
user_rated_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,id,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,50e6da19e4b0d8a78a0e9794,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Dim Sum Deluxe,57813d57498e0991ffe4b720,43.726953,-79.394260,Dim Sum Restaurant
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,5082ef77e4b0a7491cf7b022,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,50ed9da8e4b081eabee12672,43.728026,-79.382805,Bus Line
4,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,4e6e176c45dd293273b74e3c,43.712189,-79.411978,Garden


In [39]:
rating_list=[]
result_list=[]
c=0
for i in user_rated_venues['id']:
    if c<10:
        
        url1='https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
        results1 = requests.get(url1).json()#['response']['venue']['rating']
        
        result_list.append(results1)
        try:
            rating_list.append(results1['response']['venue']['rating'])
        except:
            rating_list.append("not rated")
        
    else:
        break

In [40]:
user_rated_venues['rating']=rating_list

In [41]:
final_user_rated = user_rated_venues[user_rated_venues['rating'] != 'not rated']
final_user_rated.reset_index(inplace=True)
final_user_rated=final_user_rated.drop(['index'],axis=1)

In [42]:
final_user_rated.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,id,Venue Latitude,Venue Longitude,Venue Category,rating
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,50e6da19e4b0d8a78a0e9794,43.726963,-79.394382,Park,8.5
1,Davisville North,43.712751,-79.390197,Sherwood Park,4ba011c2f964a5204a5737e3,43.716551,-79.387776,Park,9.2
2,Davisville North,43.712751,-79.390197,Summerhill Market North,4e8e73c30cd6209590ae7be4,43.715499,-79.392881,Food & Drink Shop,8
3,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,4adb2fd3f964a520c42421e3,43.712641,-79.391557,Breakfast Spot,6.8
4,Davisville North,43.712751,-79.390197,Best Western Roehampton Hotel & Suites,4b7810c3f964a52030b42ee3,43.708878,-79.390880,Hotel,6.4


In [43]:
final_user_rated_onehot = pd.get_dummies(final_user_rated[['Venue Category']], prefix="", prefix_sep="")


final_user_rated_onehot['Neighbourhood'] = final_user_rated['Neighbourhood'] 


fixed_columns = [final_user_rated_onehot.columns[-1]] + list(final_user_rated_onehot.columns[:-1])
final_user_rated_onehot.head()

,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Gourmet Shop,Greek Restaurant,Gym,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewish Restaurant,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio,Neighbourhood
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Lawrence Park
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Davisville North
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Davisville North
3,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Davisville North
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Davisville North


In [44]:
final_grouped=final_user_rated_onehot.groupby(['Neighbourhood'], sort=False).sum()

In [45]:
final_grouped.head(10)

,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Gourmet Shop,Greek Restaurant,Gym,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewish Restaurant,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Davisville North,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
"Forest Hill North, Forest Hill West",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
North Toronto West,0,0,0,0,0,0,0,1,1,2,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,2,0,0,0,0,0,0,0,0,1
"The Annex, North Midtown, Yorkville",1,1,0,0,0,1,3,0,0,3,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,1,1,2,1,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0
Davisville,0,0,0,0,1,1,2,0,0,2,0,0,3,1,1,0,0,1,1,1,1,0,0,1,2,0,0,0,1,1,2,0,1,0,3,2,0,0,0,0,2,1,1,0,0,0,0
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",1,0,1,0,0,0,0,0,0,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0


In [46]:
usermatrix = final_grouped.reset_index(drop=True)
usermatrix

,American Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Café,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Food & Drink Shop,Fried Chicken Joint,Gourmet Shop,Greek Restaurant,Gym,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewish Restaurant,Liquor Store,Mexican Restaurant,Park,Pharmacy,Pizza Place,Pub,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,1,2,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,2,0,0,0,0,0,0,0,0,1
4,1,1,0,0,0,1,3,0,0,3,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,1,1,2,1,0,0,3,0,0,0,0,0,0,0,0,0,1,0,0
5,0,0,0,0,1,1,2,0,0,2,0,0,3,1,1,0,0,1,1,1,1,0,0,1,2,0,0,0,1,1,2,0,1,0,3,2,0,0,0,0,2,1,1,0,0,0,0
6,1,0,1,0,0,0,0,0,0,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0


In [47]:
rating_df=final_user_rated[['Neighbourhood','rating']]

In [48]:
rating_df.head(15)

,Neighbourhood,rating
0,Lawrence Park,8.5
1,Davisville North,9.2
2,Davisville North,8
3,Davisville North,6.8
4,Davisville North,6.4
5,Davisville North,6
6,Davisville North,5.8
7,"Forest Hill North, Forest Hill West",8.2
8,North Toronto West,7.9
9,North Toronto West,7.7


In [49]:
rating_df.dtypes

Neighbourhood    object
rating           object
dtype: object

In [50]:
rating_df['rating']=rating_df['rating'].astype('float')

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [51]:
rating_df.dtypes

Neighbourhood     object
rating           float64
dtype: object

In [52]:
rating_grouped=rating_df.groupby('Neighbourhood', sort=False)['rating'].mean()

In [53]:
rating_df_new=pd.DataFrame(rating_grouped)

In [54]:
rating_df_new=rating_df_new.reset_index()

In [55]:
rating_df_new['rating']

0    8.500000
1    7.033333
2    8.200000
3    6.900000
4    7.326087
5    7.031250
6    6.561538
Name: rating, dtype: float64

In [56]:
l1 = pd.DataFrame(rating_df_new['rating'])
l1.to_csv('Userratings_agg.csv')

In [57]:
userProfile = usermatrix.transpose().dot(rating_df_new['rating'])

userProfile

American Restaurant              13.887625
BBQ Joint                         7.326087
Bagel Shop                        6.561538
Breakfast Spot                    7.033333
Brewery                           7.031250
Burger Joint                     21.390670
Café                             36.040761
Chinese Restaurant                6.900000
Clothing Store                    6.900000
Coffee Shop                      62.963838
Convenience Store                 6.561538
Cosmetics Shop                    7.326087
Dessert Shop                     27.993750
Diner                            13.931250
Farmers Market                    7.031250
Fast Food Restaurant              6.900000
Food & Drink Shop                 7.033333
Fried Chicken Joint              13.592788
Gourmet Shop                      7.031250
Greek Restaurant                  7.031250
Gym                               7.031250
History Museum                    7.326087
Hotel                             7.033333
Indian Rest

##### Recommendation Table

In [58]:
recommendationTable_df = ((Neig_grouped*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

Neighbourhood
Parkwoods                           0.069090
Victoria Village                    0.154879
Lawrence Heights, Lawrence Manor    0.131366
Don Mills North                     0.056820
Glencairn                           0.087853
dtype: float64

In [59]:
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
top3=pd.DataFrame(recommendationTable_df)
recommendationTable_df.head()

Neighbourhood
Fairview, Henry Farm, Oriole                       0.883235
Willowdale South                                   0.734735
Bedford Park, Lawrence Manor East                  0.666293
Bathurst Manor, Downsview North, Wilson Heights    0.464521
Flemingdon Park, Don Mills South                   0.398217
dtype: float64

In [60]:
top3=top3.reset_index()
top3.columns=['Neighbourhood','recommendation']
top3=top3.head(3)
top3

,Neighbourhood,recommendation
0,"Fairview, Henry Farm, Oriole",0.883235
1,Willowdale South,0.734735
2,"Bedford Park, Lawrence Manor East",0.666293


In [61]:
f=final_data[final_data['Neighbourhood']=='Fairview, Henry Farm, Oriole']
f.sort_values(by=['rating'],ascending=False).head(3)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,id,Venue Latitude,Venue Longitude,Venue Category,rating
55,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,The LEGO Store,4e848fbb5c5c9240de8e6a80,43.778207,-79.343483,Toy / Game Store,8.0
56,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,CF Fairview Mall,4ada3af3f964a520482021e3,43.777803,-79.344226,Shopping Mall,7.7
57,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,Michel's Baguette,4bbaa0f17421a5937311c440,43.777082,-79.344557,Bakery,7.5


In [62]:
f1=final_data[final_data['Neighbourhood']=='Willowdale South']
f1.sort_values(by=['rating'],ascending=False).head(3)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,id,Venue Latitude,Venue Longitude,Venue Category,rating
128,Willowdale South,43.77012,-79.408493,The Keg,5a35b4443abcaf37eb1a0d88,43.766579,-79.412131,Steakhouse,8.3
129,Willowdale South,43.77012,-79.408493,Starbucks,4aedfeadf964a52005d121e3,43.768192,-79.413021,Coffee Shop,7.9
131,Willowdale South,43.77012,-79.408493,Konjiki Ramen,5a02789d0a464d3112a58785,43.766998,-79.412222,Ramen Restaurant,7.9


In [63]:
f2=final_data[final_data['Neighbourhood']=='Bedford Park, Lawrence Manor East']
f2.sort_values(by=['rating'],ascending=False).head(3)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,id,Venue Latitude,Venue Longitude,Venue Category,rating
104,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Aroma Espresso Bar,502bb730e4b01590f997803d,43.735975,-79.420391,Café,8.4
105,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,Dickson Home Hardware,4c27a22a905a0f473e3b6560,43.735593,-79.420089,Hardware Store,8.1
107,"Bedford Park, Lawrence Manor East",43.733283,-79.41975,The Copper Chimney,4d796616542ab1f75eb87c41,43.736195,-79.420271,Indian Restaurant,7.6


### 4. Results section where you discuss the results:

As per above data and methodology, turists are recommended best places to visit in different neighbourhood according their preferences

### 5. Discussion section where you discuss any observations you noted and any recommendations you can make based on the results:

This model is very useful to visitors or tourists as per their preferences, which can guide them thoroughly. Th
Learns user's preferencesere is a draw back also that it can't take a glance on personal interests and economical factors.

### 6. Conclusion section where you conclude the report.

Although it is a very good recommandation system, but it is not so much precise because we also need to see some others factors. I was able to add only some of th